In [1]:
import numpy as np
from numpy import fft

from scipy.fftpack import dct, idct
from scipy.io import wavfile

from PIL import Image

from matplotlib import pyplot as plt
%matplotlib inline

Implementacija DKT po formuli

In [2]:
def u(k):
    if k == 0:
        return 1
    else:
        return 1/np.sqrt(2)

In [3]:
def my_dct(data):
    N = data.shape[0]
    coefs = np.zeros(data.shape)
    
    for k in range(N):
        part_sum = 0
        angle_part = np.pi*k/(2*N)
        for n in range(N):
            part_sum += x[n] * np.cos(angle_part*(2*n+1))
    
        coefs[k] = u(k) * part_sum 
    return coefs

Implementacija brze DKT, pomocu FFT

In [4]:
def my_dct_using_fft(data):
    N = data.shape[0]
    
    # DCT je ekivalentan DFT-u velicine 4N, ciji su parni indeksi jednaki 0 
    # Ako je nas signal 1 2 3
    # ovo pravi signal 0 1 0 2 0 3 0 3 0 2 0 1
    new_signal              = np.zeros(4 * N)
    new_signal[1 : 2*N : 2] = data
    new_signal[2*N+1 : : 2] = data[::-1]
    

    coefs = fft.fft(new_signal)[:N]
    # Vracamo samo realni deo jer DKT radi sa realnim 
    return coefs.real

Ucitavanje zvuka. Koristi se svaki 800 element, jer je my_dct izuzetno sporo radila

In [5]:
Fs, x = wavfile.read('zvuk.wav')
print(x.shape)
x = x[::800]

(2048000,)


Uporedjivanje implementacija sa scipy DKT. 

Ugradjena scipy **dct** i implementacija pomocu fft su u istoj klasi slozenosti, razlikuju se samo za konstantu 

In [6]:
timeit test = my_dct(x)

1min 1s ± 3.67 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
timeit test = dct(x)

87.3 µs ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
timeit test = my_dct_using_fft(x)

149 µs ± 7.78 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Implementacija 2D DKT preko naivne DKT, preko scipy dct i preko DKT pomocu FFT 

In [9]:
def my_dct_2(img):
    return my_dct(my_dct(img.T).T)

In [10]:
def dct_2(img):
    return dct(dct(img.T, norm='ortho').T, norm='ortho')

In [11]:
def fdct_2(img):
    M = img.shape[0]
    N = img.shape[1]
    a = np.empty([M,N],float)
    b = np.empty([M,N],float)

    for i in range(M):
        a[i,:] = my_dct_using_fft(img[i,:])
    for j in range(N):
        b[:,j] = my_dct_using_fft(a[:,j])

    return b

Ucitavanje slike i skaliranje na 64x64 sliku, opet zbog naivne implementacije DKT

In [12]:
img = Image.open('slika.jpg').convert('L')
img = img.resize((64, 64), 1)
img = np.array(img)

Uporedjivanje naivne 2D DKT, 2D DKT pomocu ugradjene dct i 2D DKT pomocu brze DKT

In [13]:
timeit test = my_dct_2(img)

74.8 ms ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
timeit test = dct_2(img)

191 µs ± 9.17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
timeit test = fdct_2(img)

2.71 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
